## Import des librairies

In [ ]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.6MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy
import spacy
import sklearn
import sklearn.metrics as metrics
import nltk
import string
import nltk.data
import re
import nltk
from string import digits
from nltk.stem import WordNetLemmatizer 
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.sparse.linalg import lobpcg
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pd.set_option('display.max_colwidth', -1)

## Import des données

In [ ]:
train = pd.read_csv('/content/gdrive/Projet_fil_rouge/data/train.csv', index_col=0)
train.head(10)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0
00025465d4725e87,"""\n\nCongratulations from me as well, use the tools well. · talk """,0,0,0,0,0,0
0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
00031b1e95af7921,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0,0,0,0,0,0
00037261f536c51d,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0,0,0,0,0,0


In [ ]:
labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

##Feature engineering et Cleaning des commentaires


###Feature engineering

In [ ]:
def is_upper(name):
    '''
      Fonction qui compte le nombre de majuscules
    '''
    count=0
    for i in name:
        if i.isupper():
            count=count+1
    return count

punct = lambda l1,l2: sum([1 for x in l1 if x in l2])

def add_features(X):
    '''
      Fonction qui ajoutes des features
    '''
    X['comment_text_len'] = X['comment_text'].apply(lambda x: len(x.split()))
    X['uppercases'] = X['comment_text'].apply(lambda x: is_upper(x)) / X.comment_text_len
    X['punctation'] = X['comment_text'].apply(lambda x: punct(x,set(string.punctuation)) ) / X.comment_text_len
    return X

In [ ]:
train = add_features(train)

In [ ]:
train.head(2)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_len,uppercases,punctation
id,,,,,,,,,,
0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0,43,0.395349,0.232558
000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0,17,0.470588,0.705882


###Cleaning des commentaires du jeu "train" labellisé

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Déclaration des stopwords
stop_words = set(stopwords.words('english'))

# Déclaration du tokenizer
tokenizer = nltk.RegexpTokenizer(r"\w+")

# Déclaration du lemmatizer
nltk.download('wordnet')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Suppression des digits
remove_digits = str.maketrans('', '', digits)

def cleaning(comment):
    '''
      Fonction qui transforme les commentaires : 
        - mise en minuscules
        - transformation des contractions (exemple : "i'm" en "i am ")
        - suppression des stopwords
        - lemmatization des mots (uniformisation des mots, par exemple transformation de "words" en "word")
    '''
    # Step 1: Transform the string into lower case words
    comment_no_digit = comment.translate(remove_digits)
    comment_lower = comment_no_digit.lower().replace('\d+', '')
    
    # Step 2: Remove or replace the unwanted expressions 
    comment_inter = re.sub(r"i'm", "i am ", comment_lower)
    comment_inter = re.sub(r"what's", "what is ", comment_inter)
    comment_inter = re.sub(r"\'ve", " have ", comment_inter)
    comment_inter = re.sub(r"\'s", " ", comment_inter)
    comment_inter = re.sub(r"can't", "can not ", comment_inter)
    comment_inter = re.sub(r"n't", " not ", comment_inter)
    comment_inter = re.sub(r"\'re", " are ", comment_inter)
    comment_inter = re.sub(r"\'d", " would ", comment_inter)
    comment_inter = re.sub(r"\'ll", " will ", comment_inter)
    comment_inter = re.sub(r"\'scuse", " excuse ", comment_inter)
    comment_inter = re.sub('\W', ' ', comment_inter)
    comment_inter = re.sub('\s+', ' ', comment_inter)
    
    tokens = tokenizer.tokenize(comment_inter)
    tokens_without_sw = [word for word in tokens if not word in stop_words]
    
    tokens_lemm = [lemmatizer.lemmatize(w) for w in tokens_without_sw]
        
    return tokens_lemm

def strg(x):
    '''
      Fonction qui reconstruit une chaîne de caratères à partir d'une liste
    '''
    return (" ".join(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
train['comment_preprocessed'] = train.comment_text.apply(lambda x: cleaning(x))

In [ ]:
train['comment_cleaned'] = train.comment_preprocessed.apply(lambda x: strg(x))

In [ ]:
train.head(2)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_len,uppercases,punctation,comment_preprocessed,comment_cleaned
id,,,,,,,,,,,,
0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0,43,0.395349,0.232558,"[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalism, closure, gas, voted, new, york, doll, fac, please, remove, template, talk, page, since, retired]",explanation edits made username hardcore metallica fan reverted vandalism closure gas voted new york doll fac please remove template talk page since retired
000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0,17,0.470588,0.705882,"[aww, match, background, colour, seemingly, stuck, thanks, talk, january, utc]",aww match background colour seemingly stuck thanks talk january utc


##Construction et évaluation du modèle TF-IDF + classifier chain


### Train/Valid split à partir du jeu "train" labellisé

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train.comment_cleaned,train[labels],test_size=0.3,random_state=0)

### Construction du modèle de Regression Logistique sur le train set

In [ ]:
#vocab_size = pd.DataFrame(train.comment_cleaned.str.split(expand=True).unstack().value_counts())
#print("Taille du vocabulaire des commentaires nettoyés : {} mots".format(vocab_size.shape[0]))

In [ ]:
tfidf_vec=TfidfVectorizer(min_df=10, max_features=8000)
tfidf=tfidf_vec.fit_transform(X_train)
X_tfidf = pd.DataFrame(tfidf.todense())
X_tfidf.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7960,7961,7962,7963,7964,7965,7966,7967,7968,7969,7970,7971,7972,7973,7974,7975,7976,7977,7978,7979,7980,7981,7982,7983,7984,7985,7986,7987,7988,7989,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.122218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_tfidf = pd.concat([X_tfidf,train[['comment_text_len','uppercases']].reset_index()], axis=1)

In [ ]:
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())

# Training logistic regression model on train data
classifier.fit(X_tfidf, Y_train)

ClassifierChain(classifier=LogisticRegression(C=1.0, class_weight=None,
                                              dual=False, fit_intercept=True,
                                              intercept_scaling=1,
                                              l1_ratio=None, max_iter=100,
                                              multi_class='auto', n_jobs=None,
                                              penalty='l2', random_state=None,
                                              solver='lbfgs', tol=0.0001,
                                              verbose=0, warm_start=False),
                order=None, require_dense=[True, True])

### Evaluation du modèle sur la validation set

In [ ]:
valid_tfidf = tfidf_vec.transform(X_valid)
valid_tfidf = pd.DataFrame(valid_tfidf.todense())

In [ ]:
valid_tfidf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7960,7961,7962,7963,7964,7965,7966,7967,7968,7969,7970,7971,7972,7973,7974,7975,7976,7977,7978,7979,7980,7981,7982,7983,7984,7985,7986,7987,7988,7989,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.170147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# predict
predictions = classifier.predict(valid_tfidf)
probabilities = classifier.predict_proba(valid_tfidf)
print("ROC AUC = ", roc_auc_score(Y_valid, probabilities.toarray(), average="macro"))
print("Accuracy = ",accuracy_score(Y_valid,predictions))

ROC AUC =  0.9706124555861809
Accuracy =  0.9212483288770054


On obtient un AUC = 0.9706 sur le validation set.